In [11]:
from torch.utils.data import DataLoader
import math
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from datasets import load_dataset, DatasetDict
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer


In [13]:
import nltk
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...


True

In [28]:
#load dataset

ds = load_dataset("KingName1/food.com")

lemm = WordNetLemmatizer()

def clean_ingredient(x):
    x = x.lower()
    x = re.sub(r"\d+\/\d+|\d+|\b(cups?|tbsps?|tablespoons?|grams?|ounces?)\b", "", x)
    x = x.replace(",", " ")
    x = lemm.lemmatize(x.strip())
    return x

def normalize_ingredient_list(record):

    ingredients = record["ingredients"]
    cleaned = [clean_ingredient(i) for i in ingredients]
    cleaned = list({i for i in cleaned if len(i) > 1})
    record['ingredients'] = ingredients
    return record


ds.map(normalize_ingredient_list)
ds = ds.remove_columns(["n_steps", "description", "n_ingredients", "calories", "total fat (PDV)", "sugar (PDV)", "sodium (PDV)", "protein (PDV)", "saturated fat (PDV)", "carbohydrates (PDV)", "embedding"])


In [36]:
ds
ds["train"]['ingredients'][0]

'winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt'

In [39]:


embed_texts = [ds['train']["ingredients"][i] for i in range(len(ds['train']))]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

model_name = "meta-llama/Llama-3.1-8B-Instruct"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

In [ ]:
def tokenize(entry):
    prompt = entry["input"] + "\n\n" + entry["output"]
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=2048)
    tokens["labels"] = tokens["input_ids"].copy()

    
    return tokens

Batches: 100%|██████████| 687/687 [01:02<00:00, 10.96it/s]


In [15]:
train_testvalid = ds['train'].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})